In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore")

# Data manipulation
# ==============================================================================
import numpy as np
import pandas as pd
from astral.sun import sun
from astral import LocationInfo
from skforecast.datasets import fetch_dataset
from feature_engine.datetime import DatetimeFeatures
from feature_engine.creation import CyclicalFeatures
from feature_engine.timeseries.forecasting import WindowFeatures

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from skforecast.plot import plot_residuals
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as poff
pio.templates.default = "seaborn"
poff.init_notebook_mode(connected=True)
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({'font.size': 8})

# Modelling and Forecasting
# ==============================================================================
import skforecast
import lightgbm
import sklearn
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFECV
from skforecast.recursive import ForecasterEquivalentDate, ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection import TimeSeriesFold, bayesian_search_forecaster, backtesting_forecaster
from skforecast.feature_selection import select_features
from skforecast.preprocessing import RollingFeatures
from skforecast.plot import calculate_lag_autocorrelation, plot_residuals
from skforecast.metrics import calculate_coverage
import shap

url_daily = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/daily_peak_transformers_dataset.csv"
df_daily = pd.read_csv(url_daily,  sep=';', encoding='latin-1')

df = df_daily

data = df.copy()
data = data.loc[data.id == 'T35']
data['Time'] = pd.to_datetime(data['datahora'], format='%Y-%m-%d')
data = data.set_index('Time')
data = data.sort_index()
data#.head(2)

In [ ]:
data.info()

In [ ]:
# Define expected frequency (daily, based on your dates)
freq = "D"

full_index = pd.date_range(
    start=data.index.min(),
    end=data.index.max(),
    freq=freq
)

data_full = data.reindex(full_index)

data_interpolated = data_full.interpolate(method="time")

data_interpolated = (
    data_interpolated
    .interpolate(method="time")
    .ffill()
    .bfill()
)

data = data_interpolated

data['datahora'] = pd.to_datetime(data['datahora'])

In [ ]:
data = data.loc[data['datahora'].dt.date > pd.to_datetime('2021-12-31').date()]


In [ ]:
# Verificação se possui os 2557 rows ou tem index faltando

start_date = data.index.min()
end_date = data.index.max()
complete_date_range = pd.date_range(start=start_date, end=end_date, freq=data.index.freq)
is_index_complete = (data.index == complete_date_range).all()
print(f"Index complete: {is_index_complete}")
print(f"Number of rows with missing values: {data.isnull().any(axis=1).mean()}")


missing = completeT_date_range.difference(data.index)
print(f"Missing timestamps: {(missing)}")


In [ ]:
data.info()

In [ ]:
data

In [ ]:
# Split data into train-val-test
end_train = '2022-12-31 23:59:00'
end_validation = '2023-12-31 23:59:00'
data_train = data.loc[: end_train, :].copy()
data_val   = data.loc[end_train:end_validation, :].copy()
data_test  = data.loc[end_validation:, :].copy()

print(f"Train dates      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Validation dates : {data_val.index.min()} --- {data_val.index.max()}  (n={len(data_val)})")
print(f"Test dates       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

In [ ]:
# Interactive plot of time series
# ==============================================================================
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_train.index, y=data_train['Smax'], mode='lines', name='Train'))
fig.add_trace(go.Scatter(x=data_val.index, y=data_val['Smax'], mode='lines', name='Validation'))
fig.add_trace(go.Scatter(x=data_test.index, y=data_test['Smax'], mode='lines', name='Test'))
fig.update_layout(
    title='Daily energy demand',
    xaxis_title="Time",
    yaxis_title="Smax",
    legend_title="Partition:",
    width=800,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(orientation="h", yanchor="top", y=1, xanchor="left", x=0.001)
)
#fig.update_xaxes(rangeslider_visible=True)
fig.show()


In [ ]:
# Annual, weekly and daily seasonality
# ==============================================================================
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 1, figsize=(8, 5), sharex=False, sharey=True)
axs = axs.ravel()

# Demand distribution by month
data['month'] = data.index.month
data.boxplot(column='Smax', by='month', ax=axs[0], flierprops={'markersize': 3, 'alpha': 0.3})
data.groupby('month')['Smax'].median().plot(style='o-', linewidth=0.8, ax=axs[0])
axs[0].set_ylabel('Smax')
axs[0].set_title('Demand distribution by month', fontsize=9)

# Demand distribution by week day
data['week_day'] = data.index.day_of_week + 1
data.boxplot(column='Smax', by='week_day', ax=axs[1], flierprops={'markersize': 3, 'alpha': 0.3})
data.groupby('week_day')['Smax'].median().plot(style='o-', linewidth=0.8, ax=axs[1])
axs[1].set_ylabel('Smax')
axs[1].set_title('Demand distribution by week day', fontsize=9)

fig.suptitle("Seasonality plots", fontsize=12)
fig.tight_layout()


##### Baseline model

In [ ]:
# Create baseline: value of the same hour of the previous day
# ==============================================================================
forecaster = ForecasterEquivalentDate(
                 offset    = pd.DateOffset(days=365),
                 n_offsets = 1
             )

# Train forecaster
# ==============================================================================
forecaster.fit(y=data.loc[:end_validation, 'Smax'])
forecaster


In [ ]:
# Backtesting
# ==============================================================================
cv = TimeSeriesFold(
        steps              = 365,
        initial_train_size = len(data.loc[:end_validation]),
        refit              = False
)
metric, predictions = backtesting_forecaster(
                          forecaster = forecaster,
                          y          = data['Smax'],
                          cv         = cv,
                          metric     = 'mean_absolute_error'
                       )
metric


#### Recursive multi-step forecasting


In [ ]:
# Create forecaster using LGBM
# ==============================================================================
window_features = RollingFeatures(stats=["mean"], window_sizes=365 * 1) # predict proximos 365 dias, movbing average dos últimos 1 anos
forecaster = ForecasterRecursive(
                 estimator       = LGBMRegressor(random_state=100, verbose=-1),
                 lags            = 365,
                 window_features = window_features
             )

# Train forecaster
# ==============================================================================
forecaster.fit(y=data.loc[:end_validation, 'Smax'])
forecaster


In [ ]:
# from xgboost import XGBRegressor
# from skforecast.preprocessing import RollingFeatures
# from skforecast.recursive import ForecasterRecursive

# window_features = RollingFeatures(
#     stats=["mean"],
#     window_sizes=365
# )

# forecaster = ForecasterRecursive(
#     estimator=XGBRegressor(
#         random_state=100,
#         verbosity=0,          # <- correto
#         n_estimators=300,
#         max_depth=6,
#         learning_rate=0.05
#     ),
#     lags=365,
#     window_features=window_features
# )

# forecaster.fit(y=data.loc[:end_validation, 'Smax'])


In [ ]:
# Backtesting
# ==============================================================================
metric, predictions = backtesting_forecaster(
                          forecaster    = forecaster,
                          y             = data['Smax'],
                          cv            = cv,
                          metric        = 'mean_absolute_error',
                          verbose       = True, # Set to False to avoid printing
                      )




In [ ]:
# Plot predictions vs real value (LGBM REGRESSOR)
# ======================================================================================
fig = go.Figure()
trace1 = go.Scatter(x=data_test.index, y=data_test['Smax'], name="test", mode="lines")
trace2 = go.Scatter(x=predictions.index, y=predictions['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Real value vs predicted in test data",
    xaxis_title="Date time",
    yaxis_title="Demand",
    width=800,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(orientation="h", yanchor="top", y=1.01, xanchor="left", x=0)
)
fig.show()

metric


The autoregressive model achieves a lower MAE than the baseline model. This means that the autoregressive model is able to predict the next day's electricity demand more accurately than the baseline model.


## Multivariate

In [ ]:
url_GD = "https://media.githubusercontent.com/media/ruanvirginio/scriptsMestrado/refs/heads/main/EntrantesGD.csv?token=AL5E5H3AX5L7Z7KTD3BB243JOI75Q"
gd = pd.read_csv(url_GD,  sep=';', encoding='latin-1')

url_clientes = "https://media.githubusercontent.com/media/ruanvirginio/scriptsMestrado/refs/heads/main/Base_Quantidade_Clientes.csv?token=AL5E5H3BXZZT6EZOGNVLC7DJOI75S"
clientes = pd.read_csv(url_clientes, sep=';', encoding='latin-1')